<a href="https://colab.research.google.com/github/sergeyman/py_nlp_nltk_1/blob/main/PMLB_SE_Ch_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Pocket
# My Applications
# In Development
# NAME		  PLATFORM		      CONSUMER KEY
# Newsfeed	Android - Mobile	96779-785f8e21498d91359ddbeb55
# Newsfeed	Android - Tablet	96779-3eb65576097636bbae7de71d
# Newsfeed	Extension		      96779-ceabd8029fdf6078580f4ccc
# Newsfeed	Windows - Desktop	96779-4bdbc0dcdfe1e3e89da9b55f
# Newsfeed	Web			          96779-f71d606f7d5a34fa3a1cb015
# Newsfeed	Mobile (other)		96779-a30d61857b2c0f2c0b6cc914
# Newsfeed	Desktop (other)		96779-fcaf090432e3b6a8a08e644c

# https://stackoverflow.com/questions/41412167/why-pocket-api-returns-403-forbidden-always
# curl -X POST -H 'Content-Type: application/json' -d '{"consumer_key":"96779-4bdbc0dcdfe1e3e89da9b55f","redirect_uri":"AppTest:authorizationFinished"}' https://getpocket.com/v3/oauth/request

# https://getpocket.com/auth/authorize?request_token=XXXXXXXX-XXXX-XXXX-XXXX-XXXX&redirect_uri=AppTest:authorizationFinished

# https://getpocket.com/auth/authorize?request_token=193code=032165bf-00d3-67be-22de-523d18&redirect_uri=AppTest:authorizationFinished
# 193code=032165bf-00d3-67be-22de-523d18

# https://www.jamesfmackenzie.com/getting-started-with-the-pocket-developer-api/
# STEP 1. CREATE AN APP IN THE POCKET DEVELOPER PORTAL, GET YOUR CONSUMER KEY
# STEP 2. OBTAIN A REQUEST TOKEN
#   curl https://getpocket.com/v3/oauth/request --insecure -X POST -H "Content-Type: application/json" -H "X-Accept: application/json" -d "{\"consumer_key\":\"1234-abcd1234abcd1234abcd1234\",\"redirect_uri\":\"http://www.google.com\"}"
#   curl https://getpocket.com/v3/oauth/request --insecure -X POST -H "Content-Type: application/json" -H "X-Accept: application/json" -d "{\"consumer_key\":\"96779-4bdbc0dcdfe1e3e89da9b55f\",\"redirect_uri\":\"http://www.google.com\"}"
#   >> 3dc5559c-9528-9188-6770-108638

# STEP 3. VISIT THE POCKET WEBSITE TO AUTHORIZE YOUR APP
#   https://getpocket.com/auth/authorize?request_token=YOUR_REQUEST_TOKEN&redirect_uri=YOUR_REDIRECT_URI    //redirect_uri=http://www.google.com
#   https://getpocket.com/auth/authorize?request_token=3dc5559c-9528-9188-6770-108638&redirect_uri=http://www.google.com

# STEP 4. CONVERT YOUR REQUEST TOKEN INTO A POCKET ACCESS TOKEN
#   curl https://getpocket.com/v3/oauth/authorize --insecure -X POST -H "Content-Type: application/json" -H "X-Accept: application/json" -d "{\"consumer_key\":\"1234-abcd1234abcd1234abcd1234\",\"code\":\"dcba4321-dcba-4321-dcba-4321dc\"}"
#   curl https://getpocket.com/v3/oauth/authorize --insecure -X POST -H "Content-Type: application/json" -H "X-Accept: application/json" -d "{\"consumer_key\":\"96779-4bdbc0dcdfe1e3e89da9b55f\",\"code\":\"3dc5559c-9528-9188-6770-108638\"}"
#   >> {"access_token":"82f13b3e-87aa-cfa2-891f-68c485","username":"sergeyman78@gmail.com"}


# STEP 5. MAKE AUTHENTICATED REQUESTS AGAINST THE POCKET API
#   curl https://getpocket.com/v3/get --insecure -X POST -H "Content-Type: application/json" -H "X-Accept: application/json" -d "{\"consumer_key\":\"1234-abcd1234abcd1234abcd1234\", \"access_token\":\"5678defg-5678-defg-5678-defg56\"}"
#   curl https://getpocket.com/v3/get --insecure -X POST -H "Content-Type: application/json" -H "X-Accept: application/json" -d "{\"consumer_key\":\"96779-4bdbc0dcdfe1e3e89da9b55f\", \"access_token\":\"82f13b3e-87aa-cfa2-891f-68c485\"}"



import requests
import pandas as pd
import json
pd.set_option('display.max_colwidth', 200)

## Get Stories from Pocket

In [ ]:
CONSUMER_KEY = '96779-4bdbc0dcdfe1e3e89da9b55f'  #'your_pocket_consumer_key'

In [ ]:
auth_params = {'consumer_key': CONSUMER_KEY, 'redirect_uri': 'https://twitter.com/acombs'}

In [ ]:
tkn = requests.post('https://getpocket.com/v3/oauth/request', data=auth_params)

In [ ]:
tkn.text

'code=f4d2e007-d3a2-b1f6-1263-b04bdd'

In [ ]:
ACCESS_CODE = tkn.text.split('=')[1]

In [ ]:
ACCESS_CODE

'f4d2e007-d3a2-b1f6-1263-b04bdd'

In [ ]:
URL = 'https://getpocket.com/auth/authorize?request_token=' + ACCESS_CODE + '&redirect_uri=https%3A//www.twitter.com/acombs'

In [ ]:
URL

'https://getpocket.com/auth/authorize?request_token=36b710d7-4a68-2a7e-7612-ddad1d&redirect_uri=https%3A//www.twitter.com/acombs'

In [ ]:
usr_params = {'consumer_key': CONSUMER_KEY, 'code': ACCESS_CODE}

In [ ]:
usr = requests.post('https://getpocket.com/v3/oauth/authorize', data=usr_params)

In [ ]:
usr.text

'403 Forbidden'

In [ ]:
# ACCESS_TOKEN = usr.text.split('=')[1].split('&')[0]

ACCESS_TOKEN = "82f13b3e-87aa-cfa2-891f-68c485"

In [ ]:
ACCESS_TOKEN

'82f13b3e-87aa-cfa2-891f-68c485'

### Get 'no' stories

In [ ]:
no_params = {'consumer_key': CONSUMER_KEY,
'access_token': ACCESS_TOKEN,
'tag': 'n'}

In [ ]:
no_result = requests.post('https://getpocket.com/v3/get', data=no_params)

In [ ]:
no_result.text

'{"status":1,"complete":1,"list":{"622912615":{"item_id":"622912615","resolved_id":"622912615","given_url":"https:\\/\\/sqlitebrowser.org\\/","given_title":"DB Browser for SQLite","favorite":"0","status":"0","time_added":"1617972028","time_updated":"1617972034","time_read":"0","time_favorited":"0","sort_id":0,"resolved_title":"DB Browser for SQLite","resolved_url":"https:\\/\\/sqlitebrowser.org\\/","excerpt":"DB Browser for SQLite (DB4S) is a high quality, visual, open source tool to create, design, and edit database files compatible with SQLite. DB4S is for users and developers who want to create, search, and edit databases.","is_article":"0","is_index":"1","has_video":"0","has_image":"1","word_count":"204","lang":"en","listen_duration_estimate":79},"814196634":{"item_id":"814196634","resolved_id":"814196634","given_url":"https:\\/\\/www.jamesfmackenzie.com\\/getting-started-with-the-pocket-developer-api\\/","given_title":"Getting Started With the Pocket Developer API","favorite":"0",

In [ ]:
no_jf = json.loads(no_result.text)

In [ ]:
no_jd = no_jf['list']

In [ ]:
no_jd

{'622912615': {'excerpt': 'DB Browser for SQLite (DB4S) is a high quality, visual, open source tool to create, design, and edit database files compatible with SQLite. DB4S is for users and developers who want to create, search, and edit databases.',
  'favorite': '0',
  'given_title': 'DB Browser for SQLite',
  'given_url': 'https://sqlitebrowser.org/',
  'has_image': '1',
  'has_video': '0',
  'is_article': '0',
  'is_index': '1',
  'item_id': '622912615',
  'lang': 'en',
  'listen_duration_estimate': 79,
  'resolved_id': '622912615',
  'resolved_title': 'DB Browser for SQLite',
  'resolved_url': 'https://sqlitebrowser.org/',
  'sort_id': 0,
  'status': '0',
  'time_added': '1617972028',
  'time_favorited': '0',
  'time_read': '0',
  'time_updated': '1617972034',
  'word_count': '204'},
 '814196634': {'excerpt': 'I want to automatically extract the video from my Pocketed content and sync it to my phone. To do this, I’ll need to extend Pocket via the Developer API. Here’s how to get st

In [ ]:
no_urls=[]
for i in no_jd.values():
    no_urls.append(i.get('resolved_url'))

In [ ]:
no_urls

['https://sqlitebrowser.org/',
 'http://www.jamesfmackenzie.com/getting-started-with-the-pocket-developer-api/']

In [ ]:
len(no_urls)

2

In [ ]:
no_uf = pd.DataFrame(no_urls, columns=['urls'])

In [ ]:
no_uf

,urls
0,https://sqlitebrowser.org/
1,http://www.jamesfmackenzie.com/getting-started-with-the-pocket-developer-api/


In [ ]:
no_uf = no_uf.assign(wanted = lambda x: 'n')

In [ ]:
no_uf

,urls,wanted
0,https://sqlitebrowser.org/,n
1,http://www.jamesfmackenzie.com/getting-started-with-the-pocket-developer-api/,n


### Get 'yes' stories

In [ ]:
yes_params = {'consumer_key': CONSUMER_KEY,
'access_token': ACCESS_TOKEN,
'tag': 'y'#,
#'state': 'archive'}
}

In [ ]:
yes_result = requests.post('https://getpocket.com/v3/get', data=yes_params)

In [ ]:
yes_result.text

'{"status":1,"complete":1,"list":{"3302323341":{"item_id":"3302323341","resolved_id":"3302323372","given_url":"https:\\/\\/freesoft.ru\\/blog\\/kak-v-viber-vosstanovit-perepisku-posle-udaleniya#article-title-3","given_title":"\\u041a\\u0430\\u043a \\u0432 Viber \\u0432\\u043e\\u0441\\u0441\\u0442\\u0430\\u043d\\u043e\\u0432\\u0438\\u0442\\u044c \\u043f\\u0435\\u0440\\u0435\\u043f\\u0438\\u0441\\u043a\\u0443 \\u0438 \\u0443\\u0434\\u0430\\u043b\\u0435\\u043d\\u043d\\u044b\\u0435 \\u0441\\u043e\\u043e\\u0431\\u0449\\u0435\\u043d\\u0438\\u044f","favorite":"0","status":"0","time_added":"1617972409","time_updated":"1617972414","time_read":"0","time_favorited":"0","sort_id":0,"resolved_title":"Viber: \\u0432\\u043e\\u0441\\u0441\\u0442\\u0430\\u043d\\u0430\\u0432\\u043b\\u0438\\u0432\\u0430\\u0435\\u043c \\u0443\\u0434\\u0430\\u043b\\u0435\\u043d\\u043d\\u0443\\u044e \\u043f\\u0435\\u0440\\u0435\\u043f\\u0438\\u0441\\u043a\\u0443","resolved_url":"https:\\/\\/freesoft.ru\\/blog\\/kak-v-viber-

In [ ]:
yes_jf = json.loads(yes_result.text)

In [ ]:
yes_jd = yes_jf['list']

In [ ]:
yes_jf

{'complete': 1,
 'error': None,
 'list': {'3302323341': {'excerpt': 'Пользовались мессенджером, удалили, теперь вдруг решили установить обратно и вернуть всю переписку? Все просто: делаем обычную установку или переустановку прилож',
   'favorite': '0',
   'given_title': 'Как в Viber восстановить переписку и удаленные сообщения',
   'given_url': 'https://freesoft.ru/blog/kak-v-viber-vosstanovit-perepisku-posle-udaleniya#article-title-3',
   'has_image': '1',
   'has_video': '1',
   'is_article': '1',
   'is_index': '0',
   'item_id': '3302323341',
   'lang': 'ru',
   'listen_duration_estimate': 231,
   'resolved_id': '3302323372',
   'resolved_title': 'Viber: восстанавливаем удаленную переписку',
   'resolved_url': 'https://freesoft.ru/blog/kak-v-viber-vosstanovit-perepisku-posle-udaleniya',
   'sort_id': 0,
   'status': '0',
   'time_added': '1617972409',
   'time_favorited': '0',
   'time_read': '0',
   'time_to_read': 3,
   'time_updated': '1617972414',
   'top_image_url': 'https://f

In [ ]:
yes_urls=[]
for i in yes_jd.values():
    yes_urls.append(i.get('resolved_url'))

In [ ]:
len(yes_urls)

1

In [ ]:
yes_urls

['https://freesoft.ru/blog/kak-v-viber-vosstanovit-perepisku-posle-udaleniya']

In [ ]:
yes_uf = pd.DataFrame(yes_urls, columns=['urls'])

In [ ]:
yes_uf

,urls
0,https://freesoft.ru/blog/kak-v-viber-vosstanovit-perepisku-posle-udaleniya


In [ ]:
yes_uf = yes_uf.assign(wanted = lambda x: 'y')

In [ ]:
yes_uf

,urls,wanted
0,https://freesoft.ru/blog/kak-v-viber-vosstanovit-perepisku-posle-udaleniya,y


In [ ]:
df = pd.concat([yes_uf, no_uf])

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

,urls,wanted
0,https://freesoft.ru/blog/kak-v-viber-vosstanovit-perepisku-posle-udaleniya,y
0,https://sqlitebrowser.org/,n
1,http://www.jamesfmackenzie.com/getting-started-with-the-pocket-developer-api/,n


## Download Articles to Run Through Model

In [ ]:
import urllib

In [ ]:
EMBEDLY_KEY = 'you_embedly_api_key_here'

def get_html(x):
    try:
        qurl = urllib.parse.quote(x)
        rhtml = requests.get('https://api.embedly.com/1/extract?url=' + qurl + '&key=' + EMBEDLY_KEY)
        ctnt = json.loads(rhtml.text).get('content')
    except:
        return None
    return ctnt

In [ ]:
df.loc[:,'html'] = df['urls'].map(get_html)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

### Extract the text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
def get_text(x):
    soup = BeautifulSoup(x, 'html5lib')
    text = soup.get_text()
    return text

In [ ]:
df.loc[:,'text'] = df['html'].map(get_text)

In [ ]:
df

# Implement Tfid Vectorization & Fit Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [ ]:
vect = TfidfVectorizer(ngram_range=(1,3), stop_words='english', min_df=3)

In [ ]:
tv = vect.fit_transform(df['text'])

In [ ]:
clf = LinearSVC()

In [ ]:
model = clf.fit(tv, df['wanted'])

## Pull New Articles from Google Drive Sheet to Evaluate

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
JSON_API_KEY = r'/your/path/to/api/key/file.json'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(JSON_API_KEY, scope)
gc = gspread.authorize(credentials)

In [ ]:
# must share with client_email in json api key file
ws = gc.open("NewStories")

In [ ]:
sh = ws.sheet1

In [ ]:
zd = list(zip(sh.col_values(2),sh.col_values(3), sh.col_values(4)))

In [ ]:
zd

In [ ]:
zf = pd.DataFrame(zd, columns=['title','urls','html'])

In [ ]:
zf.replace('', pd.np.nan, inplace=True)
zf.dropna(inplace=True)

In [ ]:
zf

In [ ]:
zf.loc[:,'text'] = zf['html'].map(get_text)

In [ ]:
zf.reset_index(drop=True, inplace=True)

In [ ]:
test_matrix = vect.transform(zf['text'])

In [ ]:
test_matrix

In [ ]:
results = pd.DataFrame(model.predict(test_matrix), columns=['wanted'])

In [ ]:
results

In [ ]:
rez = pd.merge(results,zf, left_index=True, right_index=True)

In [ ]:
rez

In [ ]:
for i, w, t in zip(rez[rez['wanted']=='y'].index, rez[rez['wanted']=='y']['wanted'], rez[rez['wanted']=='y']['title']):
    print(i, w, t)

In [ ]:
# those items to change to no
change_to_no = [130, 145, 148, 163, 178, 199, 219, 222, 223, 226, 235, 279, 348, 357, 427, 440, 542, 544, 546, 568, 614, 619, 660, 668, 679, 686, 740, 829]

In [ ]:
for i, w, t in zip(rez[rez['wanted']=='n'].index, rez[rez['wanted']=='n']['wanted'], rez[rez['wanted']=='n']['title']):
    print(i, w, t)

In [ ]:
# those items to change to yes
change_to_yes = [0, 9, 29, 35, 42, 71, 110, 190, 319, 335, 344, 371, 385, 399, 408, 409, 422, 472, 520, 534, 672]

In [ ]:
rez

In [ ]:
for i in rez.iloc[change_to_yes].index:
    rez.iloc[i]['wanted'] = 'y'

In [ ]:
for i in rez.iloc[change_to_no].index:
    rez.iloc[i]['wanted'] = 'n'

In [ ]:
rez

In [ ]:
df

In [ ]:
combined = pd.concat([df[['wanted', 'text']], rez[['wanted', 'text']]])

In [ ]:
combined

In [ ]:
tvcomb = vect.fit_transform(combined['text'], combined['wanted'])

In [ ]:
model = clf.fit(tvcomb, combined['wanted'])

In [ ]:
model

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open(r'/path/to/output/your/pickle/model_file.p', 'wb'))

In [ ]:
pickle.dump(vect, open(r'/path/to/your/pickle/vect_file.p', 'wb'))